In [2]:
from context import *

``

In [44]:
circuit.clifford_rotation_gate([0,2,3,0,7]).generator

-iYZ

In [49]:
circ = circuit.identity_circuit(5).take(circuit.clifford_rotation_gate([0,2,3,0,5])).take(circuit.clifford_rotation_gate([0,0,0,1,3,4]))

In [56]:
circ.compile(5)

CliffordCircuit(
  |[1,2][3,4]|)

In [57]:
circ.forward_map

CliffordMap(
  X0-> +XIIII
  Z0-> +ZIIII
  X1-> +IZZII
  Z1-> -IXZII
  X2-> -IYYII
  Z2-> +IIZII
  X3-> +IIIXI
  Z3-> -IIIYZ
  X4-> +IIIXY
  Z4-> +IIIIZ)